#Funciones y librerías

In [ ]:
# ========================================
# 1. Instalación de librerías externas
# ========================================
!pip install pmdarima==1.8.5
!pip install -q -U keras-tuner

# ========================================
# 2. Librerías base y utilidades generales
# ========================================
import warnings
import numpy as np
import pandas as pd
from tabulate import tabulate

# ========================================
# 3. Visualización
# ========================================
import matplotlib.pyplot as plt
import seaborn as sns

# ========================================
# 4. Preprocesamiento de datos
# ========================================
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

# ========================================
# 5. Modelos de regresión clásicos
# ========================================
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge

# ========================================
# 6. Modelos bayesianos y de procesos gaussianos
# ========================================
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel, Matern, ExpSineSquared, DotProduct

# ========================================
# 7. Modelos estadísticos y series temporales
# ========================================
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# ========================================
# 8. Evaluación de modelos
# ========================================
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score

# ========================================
# 9. Keras Tuner para ajuste de hiperparámetros
# ========================================
from keras_tuner import BayesianOptimization as OriginalBayesianOptimization
from keras_tuner.oracles import BayesianOptimizationOracle as OriginalBayesianOptimizationOracle
from keras_tuner import Tuner, Objective
from keras_tuner.tuners import SklearnTuner
from keras_tuner.src.engine import hyperparameters as hp_module
from keras_tuner.src.engine import oracle as oracle_module
from keras_tuner.src.engine import trial as trial_module
from keras_tuner.src.engine import tuner as tuner_module
from keras_tuner.src.api_export import keras_tuner_export

# ========================================
# 10. Configuración de warnings
# ========================================
warnings.filterwarnings("ignore", category=UserWarning, message="Non-stationary starting autoregressive parameters found")
warnings.filterwarnings("ignore", category=UserWarning, message="Non-invertible starting MA parameters found")
warnings.filterwarnings("ignore", category=ConvergenceWarning, message="Maximum Likelihood optimization failed to converge")

class CustomBayesianOptimizationOracle(OriginalBayesianOptimizationOracle):
    def __init__(self, *args, nu=0.5, **kwargs):
        self.nu = nu  # Establecer nu antes de llamar al constructor de la clase base
        super().__init__(*args, **kwargs)
        self.gpr = self._make_gpr()  # Asegurarse de que _make_gpr se llame después de establecer nu
    def _gpr_trained(self):
        if self.gpr is None:
            print("There is no GPR model trained yet.")
            return None
        else:
            return self.gpr
    def _make_gpr(self):
        return sklearn.gaussian_process.GaussianProcessRegressor(
            kernel=sklearn.gaussian_process.kernels.Matern(nu=self.nu),
            n_restarts_optimizer=20,
            normalize_y=True,
            alpha=self.alpha,
            random_state=self.seed,
        )

class CustomBayesianOptimization(Tuner):
    def __init__(
        self,
        hypermodel=None,
        objective=None,
        max_trials=10,
        num_initial_points=None,
        alpha=1e-4,
        beta=2.6,
        seed=None,
        hyperparameters=None,
        tune_new_entries=True,
        allow_new_entries=True,
        max_retries_per_trial=0,
        max_consecutive_failed_trials=3,
        nu=0.5,  # Valor por defecto para nu
        **kwargs
    ):
        oracle = CustomBayesianOptimizationOracle(
            objective=objective,
            max_trials=max_trials,
            num_initial_points=num_initial_points,
            alpha=alpha,
            beta=beta,
            seed=seed,
            hyperparameters=hyperparameters,
            tune_new_entries=tune_new_entries,
            allow_new_entries=allow_new_entries,
            max_retries_per_trial=max_retries_per_trial,
            max_consecutive_failed_trials=max_consecutive_failed_trials,
            nu=nu  # Pasar el parámetro nu
        )
        # Llama directamente al constructor de la clase base Tuner
        super().__init__(oracle=oracle, hypermodel=hypermodel, **kwargs)

class CustomSklearnBayesianOptimization(SklearnTuner):
    def __init__(
        self,
        hypermodel=None,
        objective=None,
        max_trials=10,
        num_initial_points=None,
        alpha=1e-1,
        beta=2.6,
        seed=None,
        hyperparameters=None,
        tune_new_entries=True,
        allow_new_entries=True,
        max_retries_per_trial=0,
        max_consecutive_failed_trials=3,
        nu=0.5,  # Valor por defecto para nu
        scoring=None,
        cv=None,
        **kwargs
    ):
        oracle = CustomBayesianOptimizationOracle(
            objective=objective,
            max_trials=max_trials,
            num_initial_points=num_initial_points,
            alpha=alpha,
            beta=beta,
            seed=seed,
            hyperparameters=hyperparameters,
            tune_new_entries=tune_new_entries,
            allow_new_entries=allow_new_entries,
            max_retries_per_trial=max_retries_per_trial,
            max_consecutive_failed_trials=max_consecutive_failed_trials,
            nu=nu  # Pasar el parámetro nu
        )
        # Llama directamente al constructor de la clase base SklearnTuner
        super().__init__(oracle=oracle, hypermodel=hypermodel, scoring=scoring, cv=cv, **kwargs)
class TimeSeriesPipeline:
    def __init__(self, model_type='SARIMA', seasonal=True, scaler=None, m=12):
        self.model_type = model_type
        self.scaler = scaler
        self.model = None
        self.seasonal = seasonal
        self.m = m  # Número de períodos en una temporada para SARIMA
    def fit(self, y_train):
        # Aplicar escalado si se proporciona
        if self.scaler:
            y_train = self.scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
        # Utilizar auto_arima para encontrar los mejores parámetros
        model_auto = auto_arima(
            y_train,
            seasonal=self.seasonal,
            m=self.m,
            trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True
        )
        try:
            order = model_auto.order
        except:
            model_auto = auto_arima(
            y_train,
            seasonal=self.seasonal,
            m=1,
            trace=False,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True)
        order = model_auto.order
        seasonal_order = model_auto.seasonal_order if self.seasonal else (0, 0, 0, 0)
        # Ajustar el modelo basado en el tipo especificado
        if self.model_type == 'ARIMA':
            self.model = ARIMA(y_train, order=order).fit()
        elif self.model_type == 'SARIMA':
            self.model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order).fit()
        # Guardar la información del modelo ajustado
        self.model_info = {
            'order': order,
            'seasonal_order': seasonal_order,
            'aic': self.model.aic,
            'bic': self.model.bic
        }
    def predict(self, start, end):
        # Realizar predicciones
        y_pred = self.model.predict(start=start, end=end)
        # Invertir escalado si se proporcionó
        if self.scaler:
            y_pred = self.scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
        return y_pred
# Clase TimeSeriesPipeline ya definida aquí...
def build_pipeline(hp, y_train, model_type='KRRBF', seasonal_order=(1, 1, 1, 4), X=X):
    # Estándarización
    scaler = MinMaxScaler()

    # Definición del modelo basado en el tipo
    if model_type == "KRRBF":
        model = KernelRidge(
            kernel='rbf',
            gamma=hp.Float("gamma", 1e-3, 100, default=1, sampling="log"),
            alpha=hp.Float("alpha", 1e-3, 100, default=1, sampling="log")
        )
    elif model_type == "KNN":
        model = KNeighborsRegressor(
            n_neighbors=hp.Int("n_neighbors", 1, 9, default=3)
        )
    elif model_type == "GP":
        model = GaussianProcessRegressor(
            alpha=hp.Float("alpha", 1e-5, 1e2, default=1, sampling="log"),
            kernel=RBF(length_scale=np.ones(X.shape[1]),length_scale_bounds=(1,1000))#WhiteKernel()+Matern(,nu=0.5),normalize_y=True,
        )
    elif model_type == "Linear":
        model = LinearRegression()
    elif model_type == "Lasso":
        model = Lasso(
            alpha=hp.Float("alpha", 1e-3, 1, default=1, sampling="log")
        )
    elif model_type == "Elastic":
        model = ElasticNet(
            alpha=hp.Float("alpha", 0.1, 1, sampling="log"),
            l1_ratio=hp.Float("l1_ratio", 0.1, 1,sampling="log")
        )
    elif model_type == "SGD":
        model = SGDRegressor(
            alpha=hp.Float("alpha", 1e-3, 1, default=0.01, sampling="log"),
            l1_ratio=hp.Float("l1_ratio", 0, 1, default=0.15)
        )
    elif model_type == "BayesRid":
        model = BayesianRidge(
            alpha_1=hp.Float("alpha_1", 1e-6, 1, default=1e-6, sampling="log"),
            alpha_2=hp.Float("alpha_2", 1e-6, 1, default=1e-6, sampling="log"),
            lambda_1=hp.Float("lambda_1", 1e-6, 1, default=1e-6, sampling="log"),
            lambda_2=hp.Float("lambda_2", 1e-6, 1, default=1e-6, sampling="log")
        )
    elif model_type == "RF":
        model = RandomForestRegressor(
            max_depth=hp.Int("max_depth", 1, 100, step=1, default=30),
            n_estimators=hp.Int("n_estimators", 1, 100, step=1, default=70)
        )
    elif model_type == "SVR":
        model = SVR(
            kernel=hp.Choice("kernel", ['linear', 'poly', 'rbf', 'sigmoid'], default='rbf'),
            C=hp.Float("C", 0.1, 1, default=1),
            epsilon=hp.Float("epsilon", 1e-3, 0.1, default=0.1, sampling="log")
        )
    elif model_type in ['ARIMA', 'SARIMA']:
        # Crear una instancia de TimeSeriesPipeline para ARIMA o SARIMA
        seasonal = (model_type == 'SARIMA')
        ts_pipeline = TimeSeriesPipeline(
            model_type=model_type,
            seasonal=seasonal,
            scaler=scaler,
            m=seasonal_order[3] if seasonal else 1
        )
        # Ajustar el modelo con los datos de entrenamiento
        ts_pipeline.fit(y_train)
        # Retornar el modelo ajustado y la información del modelo
        fitted_model = ts_pipeline
        model_info = ts_pipeline.model_info
        return fitted_model, model_info
    else:
        raise ValueError("Unrecognized model_type")
    # Creación del pipeline para los modelos de ML estándar
    pipeline =Pipeline([("reg", model)]) #Pipeline([('sca',scaler),("reg", model)])
    return pipeline
class ModelBuilder:
    def __init__(self, model_type, y_train, seasonal_order=(1, 1, 1, 3)):
        self.model_type = model_type
        self.seasonal_order = seasonal_order
        self.y_train = y_train
    def build_model(self, hp):
        return build_pipeline(hp, self.y_train, self.model_type, self.seasonal_order)
path='/content/drive/MyDrive/Doctorado/Optimización/'
def time_series_cv(X, y, model_type='KRRBF', s=0, Na='', seasonal_order=(1, 1, 1, 1),train_years = 1,test_months = 3,step_months = 3 ):
    tuners = []
    Cont = 0
    y_true_list = []
    y_pred_list = []
    y_std_list = []
    best_params_list = []
      # Años para entrenamiento
      # Meses para predicción
     # Meses para mover hacia adelante
    for ind in range(12, X.shape[0] - test_months + 1, step_months):
        # Definir los intervalos de entrenamiento y prueba
        X_train, y_train = X[(ind - train_years * 12)*s:ind], y[(ind - train_years * 12)*s:ind]
        X_test, y_test = X[ind:ind + test_months], y[ind:ind + test_months]
        if model_type in ['ARIMA', 'SARIMA']:
            model,best_params = ModelBuilder(model_type, y_train, seasonal_order).build_model(2)
            y_pred = model.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)
        else:
            model_builder = ModelBuilder(model_type, y_train, seasonal_order)
            tuner = CustomSklearnBayesianOptimization(
                hypermodel=model_builder.build_model,
                objective=Objective('score', direction='max'),
                max_trials=15,
                scoring=make_scorer(r2_score),
                directory='.',
                cv=None,
                project_name=f'12{Na}CV{Cont}_{model_type}'
            )
            print(Cont)
            Cont += 1
            tuner.search(X_train, y_train)
            tuners.append(tuner)
            # Obtener y evaluar el mejor modelo de este fold
            best_model = tuner.get_best_models(num_models=1)[0]
            best_params = tuner.get_best_hyperparameters()[0].values
            if model_type == 'GP':
                kernel_params = best_model.get_params()['reg'].kernel_.get_params()
                best_params.update(kernel_params)
                y_pred, y_std = best_model.predict(X_test, return_std=True)
                y_std_list.extend(y_std)
            elif model_type in  ['Lasso','Elastic']:
                # Extraer coeficientes de ElasticNet
                final_model = best_model.named_steps['reg']
                coef_params = final_model.coef_
                best_params.update({'coef_' + str(i): coef for i, coef in enumerate(coef_params)})
                y_pred = best_model.predict(X_test)
            elif model_type == 'RF':
                final_model = best_model.named_steps['reg']
                importances = final_model.feature_importances_
                best_params.update({'feature_importance_' + str(i): imp for i, imp in enumerate(importances)})
                y_pred = best_model.predict(X_test)
            else:
                y_pred = best_model.predict(X_test)
        best_params_list.append(best_params)
        y_true_list.extend(y_test)
        y_pred_list.extend(y_pred)
    # Guardar y_true, y_pred y y_std (si existe) en un archivo Excel
    results_dict = {'y_true': y_true_list, 'y_pred': y_pred_list}
    if y_std_list:
        results_dict['y_std'] = y_std_list
    y_results = pd.DataFrame(results_dict)
    y_results_file_path = f'{path}{test_months} Meses/Results/{model_type}_{s}.xlsx'
    y_results.to_excel(y_results_file_path, index=False)
    # Guardar los mejores hiperparámetros (incluyendo kernel si aplica) en un archivo Excel
    best_params_df = pd.DataFrame(best_params_list)
    best_params_file_path = f'{path}{test_months} Meses/Params/{model_type}_{s}.xlsx'
    best_params_df.to_excel(best_params_file_path, index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


#Data

In [ ]:
FILEID ="1BidLKcEXF6h-LKKWixtKmQ_g5hgvrkgd"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O datos.xlsx && rm -rf /tmp/cookies.txt
!dir

import pandas as pd
Xdata = pd.read_excel('datos.xlsx')
Xdata.head()
Xdata.fillna(Xdata.mean(), inplace=True)
Xdata.describe()



Xdata=Xdata[['Año', 'Mes', 'TGP',  'salarios reales', 'Indicador de Seguimiento a la Economía',
       'imporenta real', 'iva real',
       'Tasa de intervención de política monetaria (%)','TD']]#'TD-1','TD-2','TD-3' ,'TD']]
Xdata.rename(columns={'Año': 'Year', 'Mes':'Month', 'TGP':'GPR',
       'salarios reales':'RS', 'Indicador de Seguimiento a la Economía':'EMI',
       'imporenta real':'RIT', 'iva real':'RVT',
       'Tasa de intervención de política monetaria (%)':'MPIR','TD':'UR'},inplace=True) #'TD-1':'UR-1','TD-2':'UR-2','TD-3':'UR-3','TD':'UR'}, inplace=True)


X_new=Xdata.loc[1:,['GPR','RS','EMI','RIT','RVT','MPIR','UR']]#,
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1','UR-1']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',

#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1','UR-1']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-2','RS-2','EMI-2','RIT-2','RVT-2','MPIR-2','UR-2']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',

#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-2','RS-2','EMI-2','RIT-2','RVT-2','MPIR-2','UR-2']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-3','RS-3','EMI-3','RIT-3','RVT-3','MPIR-3','UR-3']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',

#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-3','RS-3','EMI-3','RIT-3','RVT-3','MPIR-3','UR-3']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-4','RS-4','EMI-4','RIT-4','RVT-4','MPIR-4','UR-4']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',
#Xdata=pd.DataFrame(scaler.fit_transform(Xdata),columns=Xdata.columns)
X_new=Xdata.loc[1:,['GPR-4','RS-4','EMI-4','RIT-4','RVT-4','MPIR-4','UR-4']]#'GPR','RS','EMI','RIT','RVT','MPIR',
Xdata=Xdata.iloc[:-1]
Xdata[['GPR-5','RS-5','EMI-5','RIT-5','RVT-5','MPIR-5','UR-5']]=X_new.values#'GPR-1','RS-1','EMI-1','RIT-1','RVT-1','MPIR-1',
# X_new=Xdata.loc[1:,'TD-1']
# Xdata=Xdata.iloc[:-1]
# Xdata['TD-2']=X_new.values
# X_new=Xdata.loc[1:,'TD-2']
# Xdata=Xdata.iloc[:-1]
# Xdata['TD-3']=X_new.values
print(Xdata.info())
scaler= MinMaxScaler()
Xdata.iloc[:,2:]=scaler.fit_transform(Xdata.iloc[:,2:])
X=Xdata.drop(['UR-5','Year','Month'], axis=1).values
y=Xdata['UR-5'].values

Xdata
i=['KRRBF','KNN','GP','Linear','Lasso','Elastic','SGD','BayesRid','RF','SVR']
Xdata

# Time series cross validation

##Classic Economic Models

In [ ]:
# Evaluar el modelo ARIMA
results_arima = time_series_cv(X,y, model_type='ARIMA',s=0)

In [ ]:
# Evaluar el modelo SARIMA
results_sarima = time_series_cv(X,y, model_type='SARIMA',s=0,seasonal_order=(3, 3, 3, 2))

##Machine Learning Models

In [ ]:
results_KRRBF=time_series_cv(X, y,model_type=i[0],s=0,Na='s0')

In [ ]:
results_KNN=time_series_cv(X, y,model_type=i[1],s=0,Na='s0')

In [ ]:
results_GP=time_series_cv(X, y,model_type=i[2],s=0,Na='s04')

In [ ]:
results_Linear=time_series_cv(X, y,model_type=i[3],s=0,Na='s01')

In [ ]:
results_Lasso=time_series_cv(X, y,model_type=i[4],s=0,Na='s03')

In [ ]:
##########################
results_Elastic=time_series_cv(X, y,model_type=i[5],s=0,Na='s01')

In [ ]:
results_SGD=time_series_cv(X, y,model_type=i[6],s=0,Na='s0')

In [ ]:
results_BayesRid=time_series_cv(X, y,model_type=i[7],s=0,Na='s0')

In [ ]:
results_RF=time_series_cv(X, y,model_type=i[8],s=0,Na='s01')

In [ ]:
results_SVR=time_series_cv(X, y,model_type=i[9],s=0,Na='s0')

# Sliding Window Cross-Validation

##Classic Economic Models

In [ ]:
# Evaluar el modelo ARIMA
results_arima_s = time_series_cv(X,y, model_type='ARIMA',s=1)

In [ ]:
# Evaluar el modelo SARIMA
results_sarima_s = time_series_cv(X,y, model_type='SARIMA',s=1,seasonal_order=(3, 3, 3, 2))

##Machine Learning Models

In [ ]:
results_KRRBF_s=time_series_cv(X, y,model_type=i[0],s=1,Na='s1')

In [ ]:
results_KNN_s=time_series_cv(X, y,model_type=i[1],s=1,Na='s1')

In [ ]:
results_GP_s=time_series_cv(X, y,model_type=i[2],s=1,Na='s12')

In [ ]:
results_Linear_s=time_series_cv(X, y,model_type=i[3],s=1,Na='s1')

In [ ]:
results_Lasso_s=time_series_cv(X, y,model_type=i[4],s=1,Na='s12')

In [ ]:
results_Elastic_s=time_series_cv(X, y,model_type=i[5],s=1,Na='s19')

In [ ]:
results_SGD_s=time_series_cv(X, y,model_type=i[6],s=1,Na='s1')

In [ ]:
results_BayesRid_s=time_series_cv(X, y,model_type=i[7],s=1,Na='s1')

In [ ]:
results_RF_s=time_series_cv(X, y,model_type=i[8],s=1,Na='s11')

In [ ]:
results_SVR_s=time_series_cv(X, y,model_type=i[9],s=1,Na='s1')

#Plots

In [ ]:
# Asegúrate de eliminar cualquier instancia previa de grid_search o modelos
try:
    del grid_search  # Borra el objeto grid_search si existe
    del model  # Borra el modelo si existe
except NameError:
    pass  # Si no existen, continua sin errores

# Asumiendo que tienes X, y definidos
# División de los datos sin barajar y con un 10% para test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# Definición del modelo con los hiperparámetros proporcionados
model = GaussianProcessRegressor(
    alpha=1,  # El valor inicial, puede ser modificado por GridSearchCV
    kernel=RBF(length_scale=np.ones(X.shape[1]), length_scale_bounds=(1, 1000000)),  # Kernel RBF con límites de longitud de escala
    normalize_y=True,
    random_state=42
)

# Definición del espacio de hiperparámetros a buscar
param_grid = {
    'alpha': [1e-5, 1e-3, 1e-1, 1, 10, 1e2] # Buscando en el rango de bounds del length_scale
}

# Definición de las métricas a evaluar
scoring = {
    'R2': make_scorer(r2_score),
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False)  # Invertido para que la MAE menor sea mejor
}

# Función para ejecutar GridSearchCV con diferentes folds y recopilar métricas
def run_grid_search(cv_folds):
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv_folds,
        n_jobs=-1,
        scoring=scoring,
        refit='R2',  # Refit usando R2 como métrica principal
        return_train_score=True
    )
    grid_search.fit(X_train, y_train)
    fold_r2_scores = [grid_search.cv_results_[f'split{i}_test_R2'][grid_search.best_index_] for i in range(cv_folds)]
    fold_mae_scores = [-grid_search.cv_results_[f'split{i}_test_MAE'][grid_search.best_index_] for i in range(cv_folds)]
    return fold_r2_scores, fold_mae_scores

# Ejecutar GridSearch con 5 folds y 10 folds
r2_scores_5, mae_scores_5 = run_grid_search(5)
r2_scores_10, mae_scores_10 = run_grid_search(10)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Asegúrate de que tus datos están definidos y ordenados
# Sustituye estas líneas por tus datos reales
# r2_scores_5 = np.random.rand(100)
# r2_scores_10 = np.random.rand(100)
# mae_scores_5 = np.random.rand(100) * 0.1
# mae_scores_10 = np.random.rand(100) * 0.1

data = [
    np.sort(r2_scores_5),
    np.sort(r2_scores_10),
    np.sort(mae_scores_5),
    np.sort(mae_scores_10)
]

labels = ['R²', 'R²', 'MAE', 'MAE']

def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])
    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value

def set_axis_style(ax, labels):
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_ylim(0, 1)

fig, ax = plt.subplots(figsize=(8, 4))

parts = ax.violinplot(
    data, showmeans=False, showmedians=False, showextrema=False
)

# Tonos pastel: azul para R², naranja para MAE
violin_colors = ['#aec7e8','#ffbb78','#aec7e8', '#ffbb78']  # tonos pastel

scale_factor = 0.3  # Violines delgados para evitar sobresalir

for i, (pc, color) in enumerate(zip(parts['bodies'], violin_colors)):
    pc.set_facecolor(color)
    pc.set_edgecolor('black')
    pc.set_alpha(0.9)

    # Escalar el ancho del violín
    verts = pc.get_paths()[0].vertices
    x = verts[:, 0]
    x_mean = np.mean(x)
    verts[:, 0] = (x - x_mean) * scale_factor + x_mean

# Calcular cuartiles y bigotes
quartile1, medians, quartile3 = [], [], []

for d in data:
    q1, med, q3 = np.percentile(d, [25, 50, 75])
    quartile1.append(q1)
    medians.append(med)
    quartile3.append(q3)

quartile1 = np.array(quartile1)
medians = np.array(medians)
quartile3 = np.array(quartile3)

whiskers = np.array([
    adjacent_values(d, q1, q3) for d, q1, q3 in zip(data, quartile1, quartile3)
])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]

# Posiciones X
inds = np.arange(1, len(data) + 1)

# Dibujar elementos de boxplot
# Dibujar una línea horizontal en la mediana
for i, med in enumerate(medians, start=1):
    ax.hlines(med, i - 0.01, i + 0.01, color='white', linewidth=1, zorder=10)

ax.vlines(inds, quartile1, quartile3, color='k', linestyle='-', lw=5)
ax.vlines(inds, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)

# Estilo de ejes y título
set_axis_style(ax, labels)
plt.tight_layout()
plt.ylabel('Score')
plt.savefig('CV5-10.pdf')
plt.show()


In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
def evaluate_model(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae =mean_absolute_error(y_test, y_pred)
    return mse,rmse,mape,r2,mae
# Definición de columnas y etiquetas
columnas = ['Model','MSE0', 'MSE1', 'RMSE0', 'RMSE1', 'MAPE0', 'MAPE1', 'MAE0', 'MAE1', 'R20', 'R21']
labels = ['ARIMA', 'SARIMA', 'RF', 'Lasso', 'SVR', 'Elastic', 'GP']
# Crear el DataFrame vacío
df = pd.DataFrame(columns=columnas)
# Lista para almacenar las filas
filas = []
# Bucle para procesar cada modelo
for label in labels:
    # Cargar datos
    data0 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Results/{label}_0.xlsx')
    data1 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Results/{label}_1.xlsx')
    # Extraer valores
    y_test = data0['y_true'].values
    y_pred0 = data0['y_pred'].values
    y_pred1 = data1['y_pred'].values
    # Calcular métricas
    mse0, rmse0, mape0, r20, mae0 = evaluate_model(y_test, y_pred0)
    mse1, rmse1, mape1, r21, mae1 = evaluate_model(y_test, y_pred1)
    # Crear nueva fila
    nueva_fila = pd.Series([label,mse0, mse1, rmse0, rmse1, mape0, mape1, mae0, mae1, r20, r21], index=columnas)
    # Agregar la nueva fila a la lista de filas
    filas.append(nueva_fila)
# Concatenar todas las filas en un único DataFrame
df = pd.concat(filas, axis=1).T.reset_index(drop=True)
# Mostrar el DataFrame resultante
#df.to_excel('/content/drive/MyDrive/Doctorado/Optimización/Results.xlsx')
df

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
def evaluate_model(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae =mean_absolute_error(y_test, y_pred)
    return mse,rmse,mape,r2,mae
# Definición de columnas y etiquetas
columnas = ['Model','MSE0', 'MSE1', 'RMSE0', 'RMSE1', 'MAPE0', 'MAPE1', 'MAE0', 'MAE1', 'R20', 'R21']
labels = ['ARIMA', 'SARIMA', 'RF', 'Lasso', 'SVR', 'Elastic', 'GP']

# Crear el DataFrame vacío
df = pd.DataFrame(columns=columnas)

# Lista para almacenar las filas
filas = []

# Bucle para procesar cada modelo
for label in labels:
    # Cargar datos
    data0 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Results/{label}_0.xlsx')
    data1 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Results/{label}_1.xlsx')

    # Extraer valores
    y_test = data0['y_true'].values
    y_pred0 = data0['y_pred'].values
    y_pred1 = data1['y_pred'].values

    # Calcular métricas
    mse0, rmse0, mape0, r20, mae0 = evaluate_model(y_test, y_pred0)
    mse1, rmse1, mape1, r21, mae1 = evaluate_model(y_test, y_pred1)
    # Crear nueva fila
    nueva_fila = pd.Series([label,mse0, mse1, rmse0, rmse1, mape0, mape1, mae0, mae1, r20, r21], index=columnas)

    # Agregar la nueva fila a la lista de filas
    filas.append(nueva_fila)

# Concatenar todas las filas en un único DataFrame
df = pd.concat(filas, axis=1).T.reset_index(drop=True)
# Mostrar el DataFrame resultante
#df.to_excel('/content/drive/MyDrive/Doctorado/Optimización/Results.xlsx')
df

In [ ]:
Xdata

In [ ]:
# Xdata=pd.DataFrame(scaler.inverse_transform(Xdata.values),columns=Xdata.columns)
# Xdata['Year']=Xdata['Year'].astype(np.int64)
# Xdata['Month']=Xdata['Month'].astype(np.int64)

In [ ]:
Xdata

In [ ]:
# Colores para los splits
# Colores para los splits
# Renombrar columnas para que sean compatibles con pd.to_datetime
#Xdata.rename(columns={'Año': 'year', 'Mes': 'month'}, inplace=True)

# Convertir 'year' y 'month' en un objeto de fecha y establecerlo como índice

Xdata['Fecha'] = pd.to_datetime(Xdata[['Year', 'Month']].assign(day=1))
Xdata = Xdata.set_index('Fecha')
# Añadir un punto para el año 2024
data_filtered = Xdata[(Xdata.index.month == 1) | (Xdata.index.month == 12)]
dates = data_filtered.index
annual_data = Xdata.resample('Y').mean()
years = annual_data.index
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from statsmodels.tsa.seasonal import seasonal_decompose
def obtener_indices_incrustados(tamano_total, tamano_ventana, cantidad_incrustada):
    indices = []

    # Recorremos las ventanas
    for i in range(0, tamano_total, tamano_ventana):
        indices.extend(range(i, min(i + cantidad_incrustada, tamano_total)))

    # Verificar si hay que completar con los últimos N índices
    tamano_actual = len(indices)
    tamano_deseado = (tamano_total // tamano_ventana) * cantidad_incrustada+cantidad_incrustada

    if tamano_actual < tamano_total:
        indices.extend(range(tamano_total - (tamano_deseado - tamano_actual), tamano_total))

    return indices


# Leer los datos de la base de datos
data = Xdata.copy()

# Renombrar columnas para que sean compatibles con pd.to_datetime
#data.rename(columns={'Año': 'year', 'Mes': 'month'}, inplace=True)

# Convertir 'year' y 'month' en un objeto de fecha y establecerlo como índice
data['Fecha'] = pd.to_datetime(data[['Year', 'Month']].assign(day=2))
data = data.set_index('Fecha')
annual_data = data.resample('Y').agg({'Year': 'first', 'Month': 'first', 'UR-5': 'mean'})
annual_data['Fecha'] = pd.to_datetime(annual_data[['Year', 'Month']].assign(day=1))
annual_data = annual_data.set_index('Fecha')

# Definir el número de splits y generar los splits una vez
n_splits = 70
tscv = TimeSeriesSplit(n_splits=n_splits)

# Asumiendo que y son tus datos de valor agrupados por año
y = data['UR'].values  # Reemplaza 'value' con el nombre de tu columna de datos
years = annual_data.index  # Obtener los años

# Colores para los splits
colors = plt.cm.viridis(np.linspace(0, 1, 80))

# Variables para almacenar las métricas
mse_scores = []
rmse_scores = []
mape_scores = []
r2_scores = []

# Descomposición estacional de la serie temporal
decomposition = seasonal_decompose(data['UR'], model='additive', period=12)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Crear las gráficas
fig, axs = plt.subplots(figsize=(20, 16))

# Gráfica 1: Valores medios por año y particiones usadas en test
axs.plot(data['UR-5'].index, data['UR-5'].values, color='black')
i = 0  # Inicializar el contador para los colores
i = 0  # Inicializar el contador para los colores
train_years = 1  # Años para entrenamiento
test_months = 12  # Meses para predicción
step_months = 6  # Meses para mover hacia adelante


for ind in range(12, data.shape[0] - test_months + 1, step_months):
    # Definir los intervalos de entrenamiento y prueba
    X_train, y_train = data.iloc[ind - train_years * 12:ind], y[ind - train_years * 12:ind]
    X_test, y_test = data.iloc[ind:ind + test_months], y[ind:ind + test_months]

    # Añadir la franja vertical para los períodos de test
    axs.axvspan(data.index[ind], data.index[ind + test_months - 1], facecolor=colors[i], alpha=0.3)

    i += 1  # Incrementar el índice del color

# Asegurar que los últimos 6 meses estén pintados si son el último set de predicción
axs.set_xticks(years)
axs.set_xticklabels(years.year, rotation=45)
axs.set_yscale('log')
plt.tight_layout()
pdf_path = 'Split.pdf'
plt.savefig(pdf_path)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
def obtener_indices_incrustados(tamano_total, tamano_ventana, cantidad_incrustada):
    indices = []

    # Recorremos las ventanas
    for i in range(0, tamano_total, tamano_ventana):
        indices.extend(range(i, min(i + cantidad_incrustada, tamano_total)))

    # Verificar si hay que completar con los últimos N índices
    tamano_actual = len(indices)
    tamano_deseado = (tamano_total // tamano_ventana) * cantidad_incrustada+cantidad_incrustada

    if tamano_actual < tamano_total:
        indices.extend(range(tamano_total - (tamano_deseado - tamano_actual), tamano_total))

    return indices
def plot_decomposition_and_models_combined(data, labels, highlight_periods, highlight_lines, params):
    """
    Genera una figura con 6 subplots organizados en 2 filas y 3 columnas:
    dos conjuntos de gráficas con predicciones de modelos y descomposición estacional.
    Para el modelo GP, se agrega una banda de confianza utilizando la columna y_std.

    :param data: DataFrame con los datos originales
    :param labels: Lista de listas con etiquetas de los modelos para los dos conjuntos
    :param highlight_periods: Lista de períodos a destacar con franjas
    :param highlight_lines: Lista de fechas a destacar con líneas verticales
    :param params: Lista de listas de parámetros para los dos conjuntos [periodo, cantidad_incrustada, fold]
    """

    # Crear una figura con 6 subplots en 2 filas y 3 columnas
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    def plot_individual_set(axes, row, data, labels1, labels2, highlight_periods, highlight_lines, params):
        periodo, cantidad_incrustada, fold = params
        tamano_total = len(data)
        tamano_ventana = periodo

        indices_resultantes = obtener_indices_incrustados(tamano_total, tamano_ventana, cantidad_incrustada)

        # Gráfica 1
        for label in labels1:
            data0 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/{fold}/Results/{label}_0.xlsx')
            y_pred0 = data0['y_pred'].values
            indices_resultantes = obtener_indices_incrustados(len(y_pred0), tamano_ventana, cantidad_incrustada)
            y_pred0 = y_pred0[indices_resultantes]
            if label == 'GP' and 'y_std' in data0.columns:
                y_std = data0['y_std'].values[indices_resultantes]
                lower_bound = y_pred0 - 1.96 * y_std
                upper_bound = y_pred0 + 1.96 * y_std
                #axes[row, 0].fill_between(data.index[12:-1], lower_bound, upper_bound, color='green', alpha=0.3, label=f'{label} 95% CI')

            axes[row, 0].plot(data.index[12:-1], y_pred0, label=label)

        axes[row, 0].plot(data.index[12:-1], data0['y_true'].values[indices_resultantes], color='black')

        # Ajuste del eje X para mostrar fechas completas
        axes[row, 0].set_xticks(data.index[12:-1][::int(len(data)/10)])  # Mostrar 10 fechas distribuidas en el eje X
        axes[row, 0].set_xticklabels(data.index.strftime('%Y-%m')[12:-1][::int(len(data)/10)], rotation=45)
        axes[row, 0].grid(True, linestyle='--', alpha=0.7)
        axes[row, 0].legend()
        axes[row, 0].set_yscale('log')

        for start, end in highlight_periods:
            axes[row, 0].axvspan(pd.to_datetime(start), pd.to_datetime(end), color='bisque', alpha=0.3)

        for start, end in highlight_lines:
            axes[row, 0].axvline(pd.to_datetime(start), color='black', linestyle='--')

        # Gráfica 2
        for label in labels2:
            data0 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/{fold}/Results/{label}_1.xlsx')
            y_pred0 = data0['y_pred'].values
            indices_resultantes = obtener_indices_incrustados(len(y_pred0), tamano_ventana, cantidad_incrustada)
            y_pred0 = y_pred0[indices_resultantes]
            if label == 'GP' and 'y_std' in data0.columns:
                y_std = data0['y_std'].values[indices_resultantes]
                lower_bound = y_pred0 - 1.96 * y_std
                upper_bound = y_pred0 + 1.96 * y_std
                #axes[row, 1].fill_between(data.index[12:-1], lower_bound, upper_bound, color='green', alpha=0.3, label=f'{label} 95% CI')

            axes[row, 1].plot(data.index[12:-1], y_pred0, label=label, linestyle='--')

        axes[row, 1].plot(data.index[12:-1],data0['y_true'].values[indices_resultantes], color='black', linestyle='--')

        # Ajuste del eje X para mostrar fechas completas
        axes[row, 1].set_xticks(data.index[12:-1][::int(len(data)/10)])  # Mostrar 10 fechas distribuidas en el eje X
        axes[row, 1].set_xticklabels(data.index.strftime('%Y-%m')[12:-1][::int(len(data)/10)], rotation=45)
        axes[row, 1].grid(True, linestyle='--', alpha=0.7)
        axes[row, 1].legend()
        axes[row, 1].set_yscale('log')

        for start, end in highlight_periods:
            axes[row, 1].axvspan(pd.to_datetime(start), pd.to_datetime(end), color='bisque', alpha=0.3)

        for start, end in highlight_lines:
            axes[row, 1].axvline(pd.to_datetime(start), color='black', linestyle='--')

        # Descomposición estacional
        stl = STL(data0['y_true'].values[indices_resultantes], period=periodo)
        result = stl.fit()

        seasonal = result.seasonal
        trend = result.trend - result.trend.mean()
        residual = result.resid

        axes[row, 2].plot(data.index[12:-1], trend, label='Trend', color='purple', linestyle='-')
        axes[row, 2].plot(data.index[12:-1], seasonal, label='Seasonality', color='red', linestyle='--')
        axes[row, 2].plot(data.index[12:-1], residual, label='Residual', color='blue', linestyle='-.')

        # Ajuste del eje X para mostrar fechas completas
        axes[row, 2].set_xticks(data.index[12:-1][::int(len(data)/10)])  # Mostrar 10 fechas distribuidas en el eje X
        axes[row, 2].set_xticklabels(data.index[12:-1].strftime('%Y-%m')[::int(len(data)/10)], rotation=45)
        axes[row, 2].grid(True, linestyle='--', alpha=0.7)
        axes[row, 2].legend()

        for start, end in highlight_periods:
            axes[row, 2].axvspan(pd.to_datetime(start), pd.to_datetime(end), color='bisque', alpha=0.3)

        for start, end in highlight_lines:
            axes[row, 2].axvline(pd.to_datetime(start), color='black', linestyle='--')

    # Plotea el primer conjunto de gráficas en la primera fila
    plot_individual_set(axes, 0, data, labels[0], labels[1], highlight_periods, highlight_lines, params[0])

    # Plotea el segundo conjunto de gráficas en la segunda fila
    plot_individual_set(axes, 1, data, labels[0], labels[1], highlight_periods, highlight_lines, params[1])

    plt.tight_layout()
    plt.savefig('PaperDM_combined.pdf')
    plt.show()

# Ejemplo de uso:
highlight_periods = [('2008-01-01', '2008-12-31'), ('2019-11-25', '2023-05-05')]
highlight_lines = [('2006-08-07', '2010-08-07'), ('2010-08-07', '2014-08-07'), ('2014-08-07', '2018-08-07'), ('2018-08-07', '2022-08-07'),('2022-08-07','2026-08-07')]

labels = [['ARIMA', 'Lasso', 'GP'], ['ARIMA', 'Lasso', 'GP']]
params = [[3, 3, '3 Meses'], [12, 6, '12 Meses']]
plot_decomposition_and_models_combined(Xdata.copy(), labels, highlight_periods, highlight_lines, params)



In [ ]:
Xdata

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def obtener_indices_filtrados(data, step):
    """
    Filtra los índices de un DataFrame o Series a partir del primer año,
    tomando cada 'step' datos.

    :param data: DataFrame o Series con un índice de tipo datetime
    :param step: Paso para seleccionar los datos (3 o 6, por ejemplo)
    :return: Índices filtrados
    """
    first_year = data.index[0].year
    data_after_first_year = data[data.index.year > first_year]
    selected_indices = data_after_first_year.index[::step]
    return selected_indices

def filtrar_etiquetas(selected_dates, interval):
    """
    Filtra las etiquetas para mostrar solo cada cierto número de meses.

    :param selected_dates: Lista de fechas seleccionadas
    :param interval: Intervalo de meses para filtrar las etiquetas
    :return: Fechas filtradas
    """
    return selected_dates[::interval]

def plot_dual_axis(ax1, label, y1_col, y2_col, params, yscale='linear', selected_dates=None, interval=6, highlight_periods=None):
    """
    Genera una gráfica con dos ejes y en un subplote específico.

    :param ax1: El eje principal donde se dibujará la gráfica
    :param label: El nombre del modelo (ARIMA, Elastic, GP)
    :param y1_col: Nombre de la columna para el primer eje y
    :param y2_col: Nombre de la columna para el segundo eje y
    :param params: Lista de parámetros [periodo, cantidad_incrustada, fold]
    :param yscale: Escala para los ejes y ('linear' o 'log')
    :param selected_dates: Fechas seleccionadas para etiquetar el eje X
    :param interval: Intervalo de meses para filtrar las etiquetas del eje X
    :param highlight_periods: Lista de períodos a destacar con franjas
    """
    data0 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/{params[2]}/Params/{label}_0.xlsx')
    data1 = pd.read_excel(f'/content/drive/MyDrive/Doctorado/Optimización/{params[2]}/Params/{label}_1.xlsx')
    if label=='GP':
        data0['length_scale']=np.array(data0['length_scale'].apply(lambda x: list(map(float, x.strip('[]').split()))).to_list()).mean(axis=1)

        data1['length_scale']=np.array(data1['length_scale'].apply(lambda x: list(map(float, x.strip('[]').split()))).to_list()).mean(axis=1)


    if label in ['ARIMA', 'SARIMA']:
        data0['order'] = data0['order'].astype(str)
        data0[['p', 'd', 'q']] = data0['order'].str.extract(r'\((\d+), (\d+), (\d+)\)')
        data0[['p', 'd', 'q']] = data0[['p', 'd', 'q']].astype(int)
        data0 = data0.drop(columns=['order'])

        data1['order'] = data1['order'].astype(str)
        data1[['p', 'd', 'q']] = data1['order'].str.extract(r'\((\d+), (\d+), (\d+)\)')
        data1[['p', 'd', 'q']] = data1[['p', 'd', 'q']].astype(int)
        data1 = data1.drop(columns=(['order']))
    y1 = data0[y1_col].values
    y2 = data0[y2_col].values
    y1_1 = data1[y1_col].values
    y2_1 = data1[y2_col].values
    ax1.plot(selected_dates,y1, color='blue', label=f'{y1_col} ({label})')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid(True, linestyle='--', alpha=0.7)

    ax2 = ax1.twinx()
    ax2.plot(selected_dates,y2, color='black', label=f'{y2_col} ({label})')
    ax2.tick_params(axis='y', labelcolor='black')

    ax1.set_yscale(yscale,linthresh=0.01)
    ax2.set_yscale(yscale,linthresh=0.01)


    ax1.plot(selected_dates,y1_1, color='blue', linestyle='--', label=f'{y1_col}_1 ({label})')
    ax2.plot(selected_dates,y2_1, color='black', linestyle='--', label=f'{y2_col}_1 ({label})')


    # Ajustar las etiquetas del eje X usando las fechas seleccionadas y filtradas
    if selected_dates is not None:
        filtered_dates = filtrar_etiquetas(selected_dates, interval)
        ax1.set_xticks(filtered_dates)  # Usar fechas directamente como ticks
        ax1.set_xticklabels([date.strftime('%Y-%m') for date in filtered_dates], rotation=45)

    # Resaltar períodos específicos con franjas
    if highlight_periods is not None:
        for start, end in highlight_periods:
            ax1.axvspan(pd.to_datetime(start), pd.to_datetime(end), color='bisque', alpha=0.3)
            ax2.axvspan(pd.to_datetime(start), pd.to_datetime(end), color='bisque', alpha=0.3)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

# Función para limpiar y convertir cadenas en listas solo si son strings
def limpiar_y_convertir(cadena):
    #if isinstance(cadena, str):
    cadena = cadena.replace(',,', ' ').replace(' ', ',')
    try:
        return eval(cadena)  # Convertir la cadena a una lista
    except SyntaxError:
        return []  # Retornar una lista vacía en caso de error
    #elif isinstance(cadena, list):
    #    return cadena  # Si ya es una lista, devolverla tal cual
    #else:
    #    return []  # Manejar otros tipos como prefieras
def plot_all_models(labels, y1_cols, y2_cols, yscales, params_list, data, highlight_periods=None):
    """
    Genera una figura con múltiples filas de subplots en función de la cantidad de listas en params_list.

    :param labels: Lista de nombres de los modelos
    :param y1_cols: Lista de nombres de las columnas para el primer eje y
    :param y2_cols: Lista de nombres de las columnas para el segundo eje y
    :param yscales: Lista de escalas para los ejes y
    :param params_list: Lista de listas de parámetros para cada conjunto de gráficos
    :param data: DataFrame con el índice de tipo datetime
    :param highlight_periods: Lista de períodos a destacar con franjas
    """
    num_rows = len(params_list)
    num_cols = len(labels)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6 * num_rows))

    for i, params in enumerate(params_list):
        step = params[1]  # Usar el paso especificado en los parámetros
        if i==0:
            selected_dates = obtener_indices_filtrados(data, step)[:-1]
        else:
            selected_dates = obtener_indices_filtrados(data,6)[:-2]
        interval = 6 if i == 0 else 3  # Usar 6 meses para la primera fila y 3 meses para la segunda
        for j, label in enumerate(labels):
            if num_rows > 1:
                ax = axes[i, j]
            else:
                ax = axes[j]
            plot_dual_axis(ax, label, y1_cols[j], y2_cols[j], params, yscale=yscales[j], selected_dates=selected_dates, interval=interval, highlight_periods=highlight_periods)

    plt.tight_layout()
    plt.savefig('PaperDMP.pdf')
    plt.show()

# Ejemplo de uso
labels = ['ARIMA', 'Elastic', 'GP']
y1_cols = ['p', 'l1_ratio', 'length_scale']
y2_cols = ['q', 'alpha', 'alpha']
yscales = ['symlog', 'symlog', 'symlog']
params_list = [[12, 3, '3 Meses'], [12, 6, '12 Meses']]

# Definir los períodos a resaltar con franjas
highlight_periods = [('2008-01-01', '2008-12-31'), ('2019-11-25', '2023-05-05')]

# Supongamos que este es tu DataFrame con un índice de tipo datetime
# data = pd.DataFrame(...)

plot_all_models(labels, y1_cols, y2_cols, yscales, params_list, data, highlight_periods)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Función para cargar y normalizar datos desde un archivo Excel
def cargar_y_normalizar(archivo, columna=None,s=0):
    df = pd.read_excel(archivo)
    if columna:
        df=pd.DataFrame(np.array(df[columna].apply(lambda x: list(map(float, x.strip('[]').split()))).to_list()))
        a=1/df.mean(axis=0)
        normalizado = abs((a/a.abs().sum()))
    elif i==0 or i==3:
        df = df.iloc[:, 1:]  # Seleccionar columnas necesarias cuando no se especifica columna
        normalizado = abs((df.mean(axis=0) / df.mean(axis=0).abs().sum()))  # Normalizar
    else:
        df = df.iloc[:, 2:]  # Seleccionar columnas necesarias cuando no se especifica columna
        normalizado = abs((df.mean(axis=0) / df.mean(axis=0).abs().sum()))  # Normalizar
    return normalizado.values

# Función para limpiar y convertir cadenas en listas solo si son strings
def limpiar_y_convertir(cadena):
    if isinstance(cadena, str):
        cadena = cadena.replace(',,', ' ').replace(' ', ',')
        try:
            return eval(cadena)  # Convertir la cadena a una lista
        except SyntaxError:
            return []  # Retornar una lista vacía en caso de error
    elif isinstance(cadena, list):
        return cadena  # Si ya es una lista, devolverla tal cual
    else:
        return []  # Manejar otros tipos como prefieras

# Función para graficar los datos normalizados en un subplot
import numpy as np
import matplotlib.pyplot as plt

def graficar_normalizado(normalizado, etiquetas, ax):
    ax.bar(np.arange(len(etiquetas)), normalizado, color='skyblue')
    # Estas etiquetas se asignan por defecto, pero luego
    # las “borramos” según la posición de la celda.
    ax.set_xlabel('Columns')
    ax.set_ylabel('Normalized Value')
    ax.set_title('Normalized Values by Column')
    ax.set_xticks(np.arange(len(etiquetas)))
    ax.set_xticklabels(etiquetas, rotation=90)

def procesar_y_graficar(archivos, etiquetas, titulo, columna=None, nrows=1, ncols=2):
    fig, axs = plt.subplots(nrows, ncols, figsize=(14, 6))
    # Si solo hay una fila o una columna, axs podría no ser
    # un array bidimensional, sino un array 1D o incluso un solo eje.
    # Para simplificar, convertimos todo a una lista plana de ejes.
    axs = np.array(axs).reshape(nrows, ncols)
    for idx, archivo in enumerate(archivos):
        # Calculamos en qué fila y columna estamos
        row = idx // ncols
        col = idx % ncols
        # Cargar y normalizar datos
        normalizado = cargar_y_normalizar(archivo, columna)
        # Graficar en el subplot correspondiente
        graficar_normalizado(normalizado, etiquetas, axs[row, col])
        # Si NO estamos en la última fila, quitamos el xlabel
        if row != nrows - 1:
            print(1)
            axs[row, col].set_xlabel('')
        # Si NO estamos en la primera columna, quitamos el ylabel
        if col != 0:
            axs[row, col].set_ylabel('')
    plt.suptitle(titulo)
    plt.tight_layout()
    plt.show()

import re
import pandas as pd
def renombrar(col):
    """
    Si la columna col es del tipo Prefijo(-sufijo),
    donde Prefijo está en [GPR, RS, EMI, RIT, RVT, MPIR, UR]
    y sufijo es opcionalmente un dígito (1..5),
    la renombramos siguiendo el mapeo:
       ""   ->  "5"
       "1"  ->  "4"
       "2"  ->  "3"
       "3"  ->  "2"
       "4"  ->  "1"
       "5"  ->  ""
    De lo contrario, devolvemos la misma columna (ej. Year, Month).
    """
    pattern = r'^(GPR|RS|EMI|RIT|RVT|MPIR|UR)(?:-(\d))?$'
    m = re.match(pattern, col)
    if m:
        prefijo = m.group(1)              # p.e. "GPR", "RS", etc.
        sufijo_original = m.group(2) or ""  # p.e. "1", "2", ..., ó "" si no había

        # Tabla de mapeo
        mapa_sufijos = {
            "":  "5",
            "1": "4",
            "2": "3",
            "3": "2",
            "4": "1",
            "5": ""
        }

        sufijo_nuevo = mapa_sufijos[sufijo_original]

        if sufijo_nuevo == "":
            return prefijo       # p.e. "GPR" si quedó sin sufijo
        else:
            return f"{prefijo}-{sufijo_nuevo}"
    else:
        return col

# --- Aplicamos el renombrado ---
Xdata= Xdata.rename(columns=renombrar)
# Definir las etiquetas
try:
    b = Xdata.drop(['UR', 'Year', 'Month'], axis=1).columns
except:
    b = Xdata

# Archivos y configuraciones de 12 meses y 3 meses
archivos_12_meses = [

    ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/Lasso_0.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/RF_0.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/GP_0.xlsx', 'length_scale'),

    ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/Lasso_1.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/RF_1.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/GP_1.xlsx', 'length_scale')
]

archivos_3_meses = [

    ('/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Params/Lasso_0.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Params/RF_0.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Params/GP_0.xlsx', 'length_scale'),
    ('/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Params/Lasso_1.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Params/RF_1.xlsx', None),
    ('/content/drive/MyDrive/Doctorado/Optimización/3 Meses/Params/GP_1.xlsx', 'length_scale'),
]
fig, axs = plt.subplots(2, 3, figsize=(18, 12))
names=['Lasso','Random forest', 'Gaussian process']
for i, (archivo, columna) in enumerate(archivos_3_meses):
    # Calcula la posición en la grilla
    row = i // 3
    col = i % 3
    # Carga y normaliza
    normalizado = cargar_y_normalizar(archivo, columna, s=i)
    # Dibuja en el subplot [row, col]
    graficar_normalizado(normalizado, b, axs[row, col])
    # Quitar el X-label si NO estamos en la última fila
    if row != 0:  # 1 es la última fila (con nrows=2)
        axs[row, col].set_title('')
    else:
        axs[row, col].set_title(names[i])
    axs[row, col].set_xlabel('')
    # Quitar el Y-label si NO estamos en la primera columna
    #if col != 0:
    axs[row, col].set_ylabel('')
fig.supylabel('Feature relevance', x=-0.001, fontsize=12)

plt.tight_layout()
plt.savefig('Inter3M.pdf')
plt.show()




In [ ]:

#plt.xticklabels(columna, rotation=90)

In [ ]:
b

In [ ]:
axs=plt.figure(figsize=(10, 4))
archivo, columna =  ('/content/drive/MyDrive/Doctorado/Optimización/12 Meses/Params/GP_1.xlsx', 'length_scale')
normalizado = cargar_y_normalizar(archivo, columna, s=i)
plt.bar(np.arange(len(b)), normalizado, color='skyblue')
# Estas etiquetas se asignan por defecto, pero luego
# las “borramos” según la posición de la celda.
plt.yticks([])
plt.xlabel('Feature')
plt.xticks(np.arange(len(b)),labels=b, rotation=90,fontsize=16)
plt.tight_layout()
plt.show()
#plt.xticklabels(columna, rotation=90)

In [ ]:
Xdata.columns

In [ ]:
# Graficar los datos de 12 meses en una figura
fig, axs = plt.subplots(2, 3, figsize=(18, 12))
for i, (archivo, columna) in enumerate(archivos_12_meses):
    # Calcula la posición en la grilla
    row = i // 3
    col = i % 3
    # Carga y normaliza
    normalizado = cargar_y_normalizar(archivo, columna, s=i)
    # Dibuja en el subplot [row, col]
    graficar_normalizado(normalizado, b, axs[row, col])
    # Quitar el X-label si NO estamos en la última fila
    if row != 0:  # 1 es la última fila (con nrows=2)
        axs[row, col].set_title('')
    else:
        axs[row, col].set_title(names[i])
    axs[row, col].set_xlabel('')
    # Quitar el Y-label si NO estamos en la primera columna
    #if col != 0:
    axs[row, col].set_ylabel('')
fig.supylabel('Feature relevance', x=-0.001, fontsize=12)

plt.tight_layout()
plt.savefig('Inter6M.pdf')
plt.show()
